# M5のおさらい

- M5ではtreeの形成は普通のdecision treeと同じように行う（？）
    - これは作りたいモデルとはたぶん異なる。
    

# M5PP

- ふたつに分割するアルゴリズムを作る
    - これをrecursiveに適用すればcascading treeになるはず
- 分割にあたっては線形モデルのエラーを最小化するようにする
- ただし、ifelse型のスイッチは最適化困難なので、sigmoidかtanhを用いる。
- 以下のようなmodelを小さい$a$でまずtrainし、徐々に$a$を大きくしていくのがいいだろうか？
　　$$\left[\tanh\left(a(x+b)\right) -1\right]\times\text{回帰式}$$
  
- regime switch用のファクターごとにモデルを作って比較するのが安直ではあるが、もっとスマートな方法はあるか
    - ものすごくきついL1正則項を入れればいい気がする

In [13]:
import tensorflow as tf
import numpy as np

## random data の生成

In [14]:
Nsample = 50000

In [15]:
Nfactor = 10

In [16]:
trX = np.random.randn(Nsample , Nfactor)

In [17]:
Nregime  = 2

In [18]:
A = np.random.randn(Nfactor , Nregime)

In [19]:
trZ = np.dot(trX , A)

In [20]:
Nregime_factor = 5

In [21]:
trX2 = np.random.randn(Nsample, Nregime_factor)

In [22]:
A2 = np.random.randn(Nregime_factor , 1)

In [23]:
trZ2 = np.dot(trX2 , A2)

M5やM5Pに倣い、regime switcherとしては、候補とするファクターひとつのみが含まれる不等式をカスケード式に用いることを想定

従って


In [24]:
trY = np.array([z[0]  if  trZ2[i , 0] > 0 else z[1] for i , z in enumerate(trZ)])

In [25]:
ybias = np.mean(trY)

In [26]:
yscaler = np.std(trY)

## 標準化しない場合

In [27]:
trY2 = trY

## 標準化する場合

In [16]:
trY2 = (trY - ybias ) / yscaler

In [17]:
np.mean(trY2)

-1.3713474800169934e-17

In [18]:
np.std(trY2)

1.0

## modelの定義

In [28]:
def model(X, X2 , w1a , w1b, w2 , tanh_scaler): # this network is the same as the previous one except with an extra hidden layer + dropout
    z1a = tf.matmul(X , w1a)
    z1b = tf.matmul(X , w1b)
    z2 = tf.matmul(X2 , w2)
    h = tf.nn.tanh(tf.multiply(z2 , tanh_scaler))
    my_one = tf.constant(dtype="float" , value = 1 )
    weight1a = tf.subtract(my_one , h) 
    weight1b = tf.subtract(h , my_one)
    elementwise_mul1a = tf.multiply(weight1a , z1a)
    elementwise_mul1b = tf.multiply(weight1b , z1b)


    return z1a #elementwise_mul1a + elementwise_mul1b

In [29]:
tanh_scaler = tf.Variable(0.1 , dtype="float")

In [30]:
X = tf.placeholder("float", [None, Nfactor])
X2 = tf.placeholder("float", [None, Nregime_factor])

In [31]:
Y2 = tf.placeholder("float")

In [32]:
w1a = tf.Variable(tf.random_normal(shape=[Nfactor , 1] , stddev=0.1))

In [33]:
w1b = tf.Variable(tf.random_normal(shape=[Nfactor , 1] , stddev=0.1))

In [34]:
w2 = tf.Variable(tf.random_normal(shape=[Nregime_factor , 1] , stddev=0.1))

In [35]:
y_model = model(X , X2 , w1a , w1b , w2 , tanh_scaler)

## cost functionの定義

以下の3つは同等？：

以下は巨大なテンソルを作ってしまう？

In [27]:
cost = tf.reduce_mean(tf.square(Y2 - y_model))

In [48]:
cost = tf.losses.mean_squared_error(labels=Y2 , predictions=y_model)

In [40]:
cost = tf.reduce_mean(tf.squared_difference(y_model, Y2))

##  最適化方式の指定（？）

In [49]:
train_op = tf.train.GradientDescentOptimizer(0.05).minimize(cost) # construct an optimizer to minimize cost and fit line to my data

sandbox check用：

## run

In [50]:
gpu_options = tf.GPUOptions(allow_growth=True)
sess = tf.InteractiveSession(config=tf.ConfigProto(gpu_options=gpu_options))

In [36]:
sess = tf.Session()

In [37]:
sess.run(tf.global_variables_initializer())

In [57]:
sess.run(cost , feed_dict={X:trX , X2:trX2 , Y2 : trY2})

ResourceExhaustedError: OOM when allocating tensor with shape[50000,50000] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[Node: mean_squared_error/SquaredDifference = SquaredDifference[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:GPU:0"](add, _arg_Placeholder_2_0_2/_87)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[Node: mean_squared_error/num_present/broadcast_weights/assert_broadcastable/is_valid_shape/has_valid_nonscalar_shape/has_invalid_dims/ExpandDims_1/_123 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_152_m...pandDims_1", tensor_type=DT_INT32, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]
Hint: If you want to see a list of allocated tensors when 

In [ ]:
sess.run(y_model , feed_dict={X:trX , X2:trX2 , Y2 : trY2}).shape

In [40]:
sess.run(Y2 , feed_dict={X:trX , X2:trX2 , Y2 : trY2}).shape

(50000,)

In [61]:
sess.run(Y2 - y_model , feed_dict={X:trX , X2:trX2 , Y2 : trY2}).shape

ResourceExhaustedError: OOM when allocating tensor with shape[50000,50000] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[Node: sub_1 = Sub[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:GPU:0"](_arg_Placeholder_2_0_2/_171, add)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


Caused by op 'sub_1', defined at:
  File "C:\Users\tsnoa\Anaconda3\envs\tensorflow\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\tsnoa\Anaconda3\envs\tensorflow\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\tsnoa\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\tsnoa\Anaconda3\envs\tensorflow\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()

### batchを切った場合

In [53]:
batch_size = 10000
n_epoch = 50

In [54]:
Nsample = trX.shape[0]

In [56]:
for i in range(n_epoch):
    for z in zip(range(0 , Nsample , batch_size) , range(batch_size , Nsample + 1 , batch_size)):
        sess.run(train_op , feed_dict={X:trX[z[0]:z[1]] , X2:trX2[z[0]:z[1]] , Y2 : trY2[z[0]:z[1]]})
        #print(z)
    #run_cost = sess.run(cost , feed_dict={X:trX , X2:trX2 , Y2 : trY2})
    #print(run_cost)
    
    #sess.run(train_operation , feed_dict={X:trX[z[0]:z[1]] , Y : trY[z[0]:z[1]]})
    #predicted_decoded_teY = sess.run(tf.argmax(py_x, 1) , feed_dict = {X : teX})
    #run_precision = np.mean(decoded_teY == predicted_decoded_teY)
    #print(i , run_precision)

ResourceExhaustedError: OOM when allocating tensor with shape[50000,50000] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[Node: mean_squared_error/SquaredDifference = SquaredDifference[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:GPU:0"](add, _arg_Placeholder_2_0_2/_87)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[Node: mean_squared_error/num_present/broadcast_weights/assert_broadcastable/is_valid_shape/has_valid_nonscalar_shape/has_invalid_dims/ExpandDims_1/_123 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_152_m...pandDims_1", tensor_type=DT_INT32, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]
Hint: If you want to see a list of allocated tensors when 

In [39]:
trY2[z[0]:z[1]].shape

(10000,)

tanh_scaler(が十分にちいさくなければ？)の微妙な変化はcostに影響を与えないだろう・・


### 一度に全部のtraining setを使った場合

In [ ]:
for i in range(100):
    sess.run(train_op , feed_dict={X:trX , X2:trX2 , Y2 : trY2})
    #print(sess.run(w1a)[0,0])
    print(sess.run(tanh_scaler))
    #print(sess.run(cost , feed_dict={X:trX , X2:trX2 , Y2 : trY2}))

In [ ]:
sess.run(w1b)

# sandbox

In [ ]:
sess.run(tanh_scaler)

In [ ]:
def model(X, X2 , w1a , w1b, w2): # this network is the same as the previous one except with an extra hidden layer + dropout
    z1a = tf.matmul(X , w1a)
    z1b = tf.matmul(X , w1b)
    z2 = tf.matmul(X2 , w2)
    h = tf.nn.tanh(z2)
    my_one = tf.constant(dtype="float" , value = 1 )
    weight1a = my_one 
    #weight1b = tf.subtract(h , my_one)
    elementwise_mul1a = tf.multiply(weight1a , z1a)
    #elementwise_mul1b = tf.multiply(weight1b , z1b)


    return elementwise_mul1a# + elementwise_mul1b

In [ ]:
sess.run(w1a)

In [ ]:
sess.run(w1a)

In [ ]:
y = sess.run(py , feed_dict={X:trX , X2:trX2})

In [ ]:
trY.shape

In [ ]:
y.shape

In [ ]:
run_cost.shape

In [ ]:
cost = tf.add(tf.constant(dtype="float" , value = 100 ) , tf.reduce_mean(tf.square(Y - y_model)))

## pattern 1 : obsolete

In [ ]:
trY = 2 * trX + np.random.randn(*trX.shape) * 0.33 # create a y value which is approximately linear but with some random noise

In [ ]:
trY = [z[0] + z[1] if  trZ2[i , 0] > 0 else z[0] - z[1] for i , z in enumerate(trZ)]

In [ ]:
def model(X, X2 , w, w2): # this network is the same as the previous one except with an extra hidden layer + dropout
    z = tf.matmul(X , w)
    z2 = tf.matmul(X2 , w2)
    h = tf.nn.tanh(z2)
    my_one = tf.constant(dtype="float" , value = 1 )
    weight = tf.subtract(my_one , h) 
    #weight2 = tf.subtract()
    #r = tf.matmul(tf.subtract(my_one , h) , z)
    elementwise_mul = tf.multiply(weight , z)


    return elementwise_mul

In [ ]:
w = tf.Variable(tf.random_normal(shape=[Nfactor , Nregime] , stddev=0.1))

In [ ]:
py = model(X , X2 , w , w2)